### Neural Networks and Sequential Modeling of Traceroutes

Treating the traceroute as a sequence of steps towards a destination could reveal more nuanced patterns. Accuracy with these models can be as high as 80% but the hypothesis that additional information may be encoded in the order was disproven as the LSTM had no additional value when compared with the standard neural nets.  Therefore, the only thing that matters is which subnets are used, not necessarily in which order.



In [1]:
import json
import pprint
import os
import pickle
import random
import sys

import matplotlib.pyplot as plt
import socket, struct
import multiprocessing as mp

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers, backend, models

from sklearn import metrics
from sklearn import datasets, cluster
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [2]:
f= open('BigFtaglOrdered.pickle', 'rb') 
ftagl=pickle.load(f)
f= open('BigExpanded.pickle', 'rb') 
expanded_routes=pickle.load(f)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator FeatureAgglomeration from version 0.21.2 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


### Create data structures (skip if serializied)

In [ ]:
X=np.zeros(shape=[len(expanded_routes),30,50])

In [ ]:
## Transform raw route information into LSTM structure with dimensionality reduction
## Note: Feature Agglomeration was done on all days

row=0
numEmpty=0
df_rows=np.zeros(shape=(len(expanded_routes),2874))
for seq in expanded_routes:
    vec=np.zeros(shape=[30,2874])
    df_vec=np.zeros(shape=[1,2874])
    there=0
    for i in range(30):
        if seq.get(i):
            vec[i][seq[i]]=1
            df_vec[0][seq[i]]=1
            there+=1

            
    vec=ftagl.transform(vec)
    df_rows[row]=df_vec
    X[row]=vec
    row+=1

In [ ]:
#ftagl.fit(df_rows)
#with open('BigFtaglOrdered.pickle', 'wb') as output:
#    pickle.dump(ftagl,output,pickle.HIGHEST_PROTOCOL)

In [ ]:
df=pd.DataFrame(df_rows)
del df_rows
del expanded_routes
df=pd.concat([pd.read_parquet('BigExtractedFeatures.parquet'),df], axis=1)

In [ ]:
df.columns=[str(x) for x in df.columns]
df.to_parquet('universe.parquet')

In [ ]:
np.savez('X',X)

### Load serialized data structures

In [2]:
df=pd.read_parquet('universe.parquet')
df=df.drop_duplicates()
df=df[df['NumHops']>2]
df=df.reset_index(drop=True)
df['Benign'].value_counts()

False    442357
True      22255
Name: Benign, dtype: int64

In [ ]:
df['Benign'].value_counts()

#### Separate temporally if required

In [ ]:
back_half=df[df.Benign == False]
back_half=back_half[round(0.8*len(back_half)):]
good_len=round(0.8*len(df[df.Benign==True]))
back_half=pd.concat([back_half,df[df.Benign== True][good_len:]],axis=0)
print(back_half['Benign'].value_counts())

front_half=df[df.Benign == False]
front_half=front_half[0:round(0.8*len(front_half))]
front_half=pd.concat([front_half,df[df.Benign == True][0:good_len]],axis=0)
print(front_half['Benign'].value_counts())

## Keras Neural Net

In [ ]:
# Dimensionality reduction if desired

# Fit new dim reduction matrix, not necessary if *ftagl*.pickle exists
#ftagl = cluster.FeatureAgglomeration(n_clusters=100)

reduced = ftagl.transform(df[df.columns.difference([header for header in df.columns if not (str(header).isdigit())])])

In [3]:
# Balance data which is NOT divided temporally

#df=df.drop(columns=df.columns.difference([header for header in df.columns if not (str(header).isdigit())]))
#reduced=pd.DataFrame(reduced)
#df=pd.concat([df,reduced],axis=1)
#del reduced


bd=df[df['Benign'] == True]
even=len(bd)
md=df[df['Benign'] == False]
md=md.reset_index(drop=True)

bd=bd.sample(frac=1).reset_index(drop=True)
md=md.sample(frac=1).reset_index(drop=True)
md=md.loc[md.index < even]
ad=pd.concat([bd,md])

del bd
del md

In [ ]:
# Balance temporally divided data

bd=front_half[front_half['Benign'] == True]
even=len(bd)
md=front_half[front_half['Benign'] == False]
md=md.reset_index(drop=True)

bd=bd.sample(frac=1).reset_index(drop=True)
md=md.sample(frac=1).reset_index(drop=True)
md=md.loc[md.index < even]
ad=pd.concat([bd,md])

bd_v=back_half[back_half['Benign'] == True]
even=len(bd_v)
md_v=back_half[back_half['Benign'] == False]
md_v=md_v.reset_index(drop=True)

bd_v=bd_v.sample(frac=1).reset_index(drop=True)
md_v=md_v.sample(frac=1).reset_index(drop=True)
md_v=md_v.loc[md_v.index < even]
ad_v=pd.concat([bd_v,md_v])

del bd
del md
del bd_v
del md_v

In [5]:
# Select feature columns and shuffle dataframe
exclude=ad.columns.difference(['indicator','Benign','Dest','Route','index'])
ad=ad.sample(frac=1).reset_index(drop=True)

In [8]:
#Model Architecture

model = tf.keras.Sequential([
# Adds a densely-connected layer with 64 units to the model:
layers.Dense(64, activation='relu', input_dim=2879),

layers.Dense(64, activation='relu'),

layers.Dense(64, activation='relu'),
# Add a sigmoid layer for outputs:
layers.Dense(1, activation='sigmoid')])

In [9]:
model.compile(optimizer=tf.train.AdamOptimizer(0.0001),
              loss='binary_crossentropy', 
              metrics=['accuracy'])

#cease=keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto', baseline=None)

# Train with temporally divided data
#history=model.fit(ad[exclude],ad['Benign'], epochs=100, verbose=1, batch_size=1024,
#         validation_data=(ad_v[exclude],ad_v['Benign']), shuffle=True)

# Train with data NOT divided temporally
history=model.fit(ad[exclude],ad['Benign'], epochs=25, verbose=1, batch_size=64,
          validation_split=0.2, shuffle=True)


Train on 35608 samples, validate on 8902 samples
Epoch 1/25
35608/35608 [==============================] - 46s 1ms/step - loss: 0.6133 - acc: 0.6865 - val_loss: 0.5797 - val_acc: 0.7074
Epoch 2/25
35608/35608 [==============================] - 45s 1ms/step - loss: 0.5428 - acc: 0.7250 - val_loss: 0.5134 - val_acc: 0.7438
Epoch 3/25
35608/35608 [==============================] - 47s 1ms/step - loss: 0.4848 - acc: 0.7678 - val_loss: 0.4739 - val_acc: 0.7678
Epoch 4/25
35608/35608 [==============================] - 45s 1ms/step - loss: 0.4522 - acc: 0.7852 - val_loss: 0.4535 - val_acc: 0.7844
Epoch 5/25
35608/35608 [==============================] - 46s 1ms/step - loss: 0.4329 - acc: 0.7952 - val_loss: 0.4428 - val_acc: 0.7887
Epoch 6/25
35608/35608 [==============================] - 45s 1ms/step - loss: 0.4189 - acc: 0.8034 - val_loss: 0.4538 - val_acc: 0.7815
Epoch 7/25
35608/35608 [==============================] - 46s 1ms/step - loss: 0.4088 - acc: 0.8087 - val_loss: 0.4267 - val_acc:

In [10]:
print("Classification report for classifier %s\n"
      % (metrics.classification_report(ad['Benign'], model.predict(ad[exclude])>.5)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(ad['Benign'], model.predict(ad[exclude]) > .5,labels=[True,False]))

Classification report for classifier               precision    recall  f1-score   support

       False       0.79      0.89      0.84     22255
        True       0.88      0.76      0.81     22255

    accuracy                           0.83     44510
   macro avg       0.83      0.83      0.82     44510
weighted avg       0.83      0.83      0.82     44510


Confusion matrix:
[[16884  5371]
 [ 2410 19845]]


In [9]:
del ad

Index(['indicator', 'Benign', 'Timeouts', 'AveragePing', 'NumHops',
       'Tail Timeouts', 'Dest', 'Reached', '0', '1',
       ...
       '2864', '2865', '2866', '2867', '2868', '2869', '2870', '2871', '2872',
       '2873'],
      dtype='object', length=2882)

## Keras LSTM 

#### I think the shortfalls of the LSTM actually have to do with the dimensionality reduction which is absolutely necessary with this quantity of data.  

### Data Generator

In [3]:
# Define a data generator to prevent memory overflow and increase training speed

class DataGenerator(keras.utils.Sequence):

    def __init__(self, data, labels, dimReduction, batch_size=64, dim=(30,50), n_classes=2, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.expanded_routes=data
        self.benign_labels=labels
        self.index=0
        self.dimReduction=dimReduction
        self.on_epoch_end()

    def __len__(self):

        return int(np.floor(len(self.expanded_routes) / self.batch_size))

    def __getitem__(self,ignore=True):

        X=np.zeros(shape=[self.batch_size,30,2874])
        row=0
        for seq in expanded_routes[self.index:self.index+self.batch_size]:
            vec=np.zeros(shape=[30,2874])
           
            for i in range(30):
                if seq.get(i):
                    vec[i][seq[i]]=1
                    
            X[row]=vec
            row+=1
            
        self.index+=self.batch_size
        return X, self.benign_labels[self.index-self.batch_size:self.index]

    def on_epoch_end(self):

        if self.shuffle == True:
            self.expanded_routes.reset_index(drop=True,inplace=True)
            np.random.seed(387562875)
            np.random.shuffle(self.expanded_routes)
            np.random.seed(387562875)
            np.random.shuffle(self.benign_labels)
        
        self.index=0

    def __data_generation(self,rand=None):

        # Initialization
        X=np.random.rand(self.batch_size,30,50)
        y = np.round(np.random.rand((self.batch_size)))

        return X, y

#### Foloowing 4 data preparation steps are unnecessary if using data generator

In [ ]:
df=pd.read_parquet('universe.parquet')
keep=df['NumHops']>2

In [ ]:
## Define empty training vars to be populated later

X=np.load('X.npz')['arr_0']
X=X[keep]
Y=np.array(df['Benign'])
Y=Y[keep]
X_e=np.array(df[['Timeouts','AveragePing','NumHops','Tail Timeouts','Reached']])
X_e=X_e[keep]
df=df[keep]
df=df.reset_index(drop=True)
print(X.shape, Y.shape, X_e.shape, df.shape)

In [ ]:
## Map benign and malicious entries for balancing purposes

benignIndices=[]
maliciousIndices=[]
for x in range(0,len(df)):
    if df['Benign'][x] == True:
        benignIndices.append(x)
    else:
        maliciousIndices.append(x)

In [ ]:
## Create training data from equal parts benign and malicious data, shuffle to avoid proximity effects

X_benign_e=X_e[benignIndices,:]
X_benign=X[benignIndices,:]
Y_benign=Y[benignIndices]
X_mal=X[maliciousIndices,:]
X_mal_e=X_e[maliciousIndices,:]
Y_mal=Y[maliciousIndices]
del X
del Y
np.random.seed(387562875)
np.random.shuffle(X_mal)
np.random.seed(387562875)
np.random.shuffle(X_mal_e)
np.random.seed(387562875)
np.random.shuffle(Y_mal)
X_mal_e=X_mal_e[0:len(X_benign),:]
X_mal=X_mal[0:len(X_benign),:]
Y_mal=Y_mal[0:len(X_benign)]
X_t_e=np.concatenate((X_benign_e,X_mal_e), axis=0)
X_t=np.concatenate((X_benign,X_mal), axis=0)
Y_t=np.concatenate((Y_benign,Y_mal), axis=0)
del X_benign
del X_benign_e
del X_mal
del X_mal_e
del Y_benign
del Y_mal
np.random.seed(387562875)
np.random.shuffle(X_t)
np.random.seed(387562875)
np.random.shuffle(X_t_e)
np.random.seed(387562875)
np.random.shuffle(Y_t)

In [6]:
# Define LSTM

model = tf.keras.Sequential([
# Adds an LSTM layer to intake the traceroute as a sequence of stops:
layers.LSTM(64, activation='relu', input_shape=(30,2874), return_sequences=False),
# Add dense layers to further interpret results:
layers.Dense(64, activation='relu'),
    
layers.Dense(64, activation='relu'),
# Add a sigmoid output layer
layers.Dense(1, activation='sigmoid')])

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 64)                752384    
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 760,769
Trainable params: 760,769
Non-trainable params: 0
_________________________________________________________________
None


In [4]:
# Balance subnet data that will be fed into LSTM, required for data generator

tvals=pd.read_parquet('BigExtractedFeatures.parquet')['Benign']
mal_routes=pd.Series(expanded_routes)[~tvals]
np.random.shuffle(mal_routes)
mal_routes=mal_routes[0:sum(tvals)]
new_expanded=pd.Series(expanded_routes)[tvals].append(mal_routes)
del mal_routes
tvals=[True]*sum(tvals)+[False]*sum(tvals)
expanded_routes=new_expanded
del new_expanded
expanded_routes=expanded_routes.reset_index(drop=True)
np.random.seed(29837492)
np.random.shuffle(expanded_routes)
np.random.seed(29837492)
np.random.shuffle(tvals)

In [7]:
model.compile(optimizer=tf.train.AdamOptimizer(0.0001),
              loss='binary_crossentropy', 
              metrics=['accuracy'])

# Train using static data
#history=model.fit(X_t,Y_t, epochs=50, verbose=1, batch_size=64,
          #validation_split=0.2, shuffle=True)

# Train and test using generators
# *** Warning: Unresolved issues with wildly fluctuating training accuracy here
trainingGen=DataGenerator(expanded_routes,tvals,False,batch_size=256)
validationGen=DataGenerator(expanded_routes[0:1000],tvals[0:1000],False)
history=model.fit_generator(generator=trainingGen,use_multiprocessing=True,epochs=5,verbose=1)

Epoch 1/5
187/187 [==============================] - 103s 553ms/step - loss: 0.6842 - acc: 0.6414
Epoch 2/5
187/187 [==============================] - 105s 564ms/step - loss: 0.5582 - acc: 0.7190
Epoch 3/5
187/187 [==============================] - 105s 562ms/step - loss: 0.4830 - acc: 0.7639
Epoch 4/5
187/187 [==============================] - 106s 565ms/step - loss: 0.4572 - acc: 0.7776
Epoch 5/5
187/187 [==============================] - 107s 570ms/step - loss: 0.4481 - acc: 0.7805


In [ ]:
#data = np.random.random((1000, 32))
#labels = random_one_hot_labels((1000, 10))

model.evaluate(X_t, Y_t, batch_size=32)

In [ ]:
print("Classification report for classifier %s\n"
      % (metrics.classification_report(Y_t, model.predict(X_t)>.5)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(Y_t, model.predict(X_t) > .5,labels=[True,False]))

# Keras Hybrid LSTM

#### LSTM + Dense neural net which exploits all available features and processes subnets as a sequence

In [ ]:
print(X_t_e.shape, X_t.shape, Y_t.shape)

In [ ]:

#Two input sequences, the first is standard statistics about the traceroute and the second the sequence of stops 
my_inputs = layers.Input(shape=(5,),dtype='float32')
ip_inputs = layers.Input(shape=(30,50),dtype='float32')

# LSTM layer
lstm_out = layers.LSTM(64, activation='relu')(ip_inputs)

# Merge outputs from the LSTM layer with the next set of inputs
x = keras.layers.concatenate([lstm_out, my_inputs])

# Add dense layers to combine LSTM information with other data
x = layers.Dense(64, activation='relu')(x)
x = layers.Dense(64, activation='relu')(x)

# Sigmoid output layer
predictions = layers.Dense(1, activation='sigmoid')(x)


model = models.Model(inputs=[my_inputs,ip_inputs], outputs=predictions)
model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss='binary_crossentropy', 
              metrics=['accuracy'])

model.summary()
#board=keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None)

In [ ]:
model.fit([X_t_e,X_t], Y_t, epochs=20, verbose =1, batch_size=64,validation_split=0.2)

In [ ]:
del X_t
del Y_t

## Autoencoder 

#### Reduce dimensionality of subnet data
#### Failed experiment: it appears the the data is too sparse for the sutoencoder to learn anything.  The output is always the same set of 4 or 5 entries predicted as ones regardless of the input

In [ ]:
df=pd.read_parquet('universe.parquet')
df=df[df['NumHops']>2]
df=df[df.columns.difference([header for header in df.columns if not (str(header).isdigit())])]
df=np.array(df)

In [ ]:
pd.Series(df.sum(axis=1)).describe()

In [ ]:
encoding_dim = 256

# input vec
input_vec = layers.Input(shape=(2874,))

x=layers.Dense(512,activation='relu')(input_vec)

x=layers.Dense(512,activation='relu')(x)

# "encoded" is the encoded representation of the input
encoded = layers.Dense(encoding_dim, activation='relu')(input_vec)

x=layers.Dense(512,activation='relu',input_dim=(256,))(encoded)

x=layers.Dense(512,activation='relu',input_dim=(256,))(x)

# "decoded" is the reconstruction of the input
decoded = layers.Dense(2874, activation='softmax')(encoded)

# this model maps an input to its reconstruction
autoencoder = models.Model(input_vec, decoded)
autoencoder.summary()
#encoder = models.Model(input_vec, encoded)

# create a placeholder for an encoded (32-dimensional) input
#encoded_input = layers.Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
#decoder_layer = autoencoder.layers[-1]
# create the decoder model
#decoder = models.Model(encoded_input, decoder_layer(encoded_input))

In [ ]:
autoencoder.compile(optimizer=tf.train.AdamOptimizer(.001), loss='categorical_crossentropy',metrics=['accuracy'])

autoencoder.fit(df,df,
                epochs=1,
                batch_size=4096,
                shuffle=True,
                validation_data=(df,df),
                verbose=1)

In [ ]:
pred=autoencoder.predict(df)

In [ ]:
pred=pd.DataFrame(pred).drop_duplicates()
pred